In [ ]:
import os, sys
from pathlib import Path
BASE_DIR = os.getcwd()
# BASE_DIR = os.path.dirname(os.path.abspath(__file__))
ROOT_DIR = os.path.join(BASE_DIR, '..')
DATA_DIR = os.path.join(ROOT_DIR, 'waymo_open_dataset', 'dataset')
print("Root directory is {}".format(ROOT_DIR))
print("Base Directory is {}".format(BASE_DIR))
print("Data Directory is {}".format(DATA_DIR))


In [ ]:
# input arguments
verbose = True

In [ ]:
import numpy as np
import pandas as pd
import pickle
sys.path.append(ROOT_DIR)
sys.path.append(os.path.join(ROOT_DIR, 'utils')) # import Box class
sys.path.append(os.path.join(ROOT_DIR, 'waymo_open_dataset')) # import Box class

import tensorflow.compat.v1 as tf

import dataset_pb2 as open_dataset
import frame_utils
from box_util import Box

In [ ]:
# list all the segments in the folder
data_split_dir = os.path.join(DATA_DIR, 'training')
if not os.path.exists(data_split_dir):
    raise Exception("Data Path is not found")
segments_list = os.listdir(data_split_dir)
segments_list

In [ ]:
# create Dict idx to segment_id
# save id and no of frames
segments_dict_list = []

# Loop over every segment in the dataset
for idx in range(len(segments_list)):
    segment_dict = {}
    # get segment id
    segment_id = '_'.join(segments_list[idx].split('_')[:5]) # will get the ID example: 'segment-10072140764565668044_4060_000_4080_000'
    segment_dir = os.path.join(data_split_dir, segment_id)
    #create folder for the current segment
    Path(segment_dir).mkdir(parents=True, exist_ok=True)
#     os.mkdir(segment_dir) # create folder for a given semgent
    
    if verbose:
        print("Reading Segment, index: {}, ID: {}".format(idx, segment_id))
#     add segment id with the respective idx to dict 
    segment_dict['id'] = segment_id
    print("Processing segment ID: {}".format(segment_id))
    # get full path of TFRecord file
    FILENAME = os.path.join(split_dir, segments_list[idx])
    if not os.path.exists(FILENAME):
        raise Exception("File cannot be found")
    # Read TFRecord
    recorded_segment = tf.data.TFRecordDataset(FILENAME, compression_type='')
    # Loop over every frame
    frame_count = 0
    for data in recorded_segment:
        # Read the first frame only
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        
        # extract the camera images, camera projection points and range images
        (range_images, 
        camera_projections, 
        range_image_top_pose) = frame_utils.parse_range_image_and_camera_projection(frame)

        # First return of Lidar data
        points, cp_points = frame_utils.convert_range_image_to_point_cloud(
            frame,
            range_images,
            camera_projections,
            range_image_top_pose)

        # Second return of Lidar data
        points_ri2, cp_points_ri2 = frame_utils.convert_range_image_to_point_cloud(
            frame,
            range_images,
            camera_projections,
            range_image_top_pose,
            ri_index=1)

        # concatenate all LIDAR points from the 5 radars.
        points_all = np.concatenate(points, axis=0)
        points_all_ri2 = np.concatenate(points_ri2, axis=0)
        
#         print((points_all.dtype))

#         # save file on the desk
#         pickle_out = open(os.path.join(POINT_CLOUD_DIR, 'point_cloud_{}'.format(segment_id)), 'wb')
#         pickle.dump(points_all, pickle_out)
#         pickle_out.close()
#         pickle_out_ri2 = open(os.path.join(POINT_CLOUD_DIR, 'point_cloud_{}_ri2'.format(segment_id)), 'wb')
#         pickle.dump(points_all_ri2, pickle_out_ri2)
#         pickle_out_ri2.close()

#         # extracting labels and save them
#         bboxes = []
        bboxes = []
        for laser_label in frame.laser_labels:
            label = laser_label.type
            length = laser_label.box.length
            width = laser_label.box.width
            height = laser_label.box.height
            x, y, z = laser_label.box.center_x, laser_label.box.center_y, laser_label.box.center_z
            heading = laser_label.box.heading
            box = [label, length, width, height, x, y, z, heading]
            bboxes.append(box)
#             box = Box([x,y,z], [length, width, height], laser_label.box.heading, label)
#             bboxes.append(box)
        labels_arr = np.array(bboxes, dtype=np.float32)
        file_name = '_'.join([segment_id, str(frame_count)])
        np.savez_compressed(os.path.join(segment_dir, '{}.npz'.format(file_name)),pc=points_all, pc_ri2 = points_all_ri2, labels=labels_arr)
    
        
        # save a npz file contains pc_first_return, pc_second_return, bboxes


        # save the bounding boxes in their respective directory
#         pickle_out = open(os.path.join(LABEL_DIR, 'label_{}'.format(segment_id)), 'wb')
#         pickle.dump(bboxes, pickle_out)
#         pickle_out.close()
        
        frame_count += 1
        segment_dict['frame_count'] = frame_count
    segments_dict_list.append(segment_dict)
    break

# save idx2segment_id dict on desk
pickle_out = open(os.path.join(data_split_dir, 'segments_dict_list'), 'wb')
pickle.dump(segments_dict_list, pickle_out)
pickle_out.close()

In [ ]:
arr = []
for i in range(5):
    arr.append([1,2,3,4,5])
final = np.array(arr, dtype=np.float32)
final

In [ ]:
print("TEST")